In [10]:
import pandas as pd

# Load the clean dataset with explicit encoding
df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')

# Iterate through each column starting from the second column (index 1)
response_tally = []

for column in df.columns[1:]:
    responses = df[column].value_counts().reset_index()
    responses.columns = ['Response', 'Count']
    responses['Column'] = column
    response_tally.append(responses)

# Concatenate the response tally dataframes
result_df = pd.concat(response_tally, ignore_index=True)

# Save the result to an Excel file
result_df.to_excel('response_tally.xlsx', index=False)


/tmp/ipykernel_4803/2500043887.py:4: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,215,248,272,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clean_kaggle_data_2022.csv', encoding='ISO-8859-1')


KeyError: 'Q'